In [1]:
from pyspark import SparkContext

from pyspark.sql.types import *
from pyspark.sql import Row
from pyspark.sql import SQLContext

from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml.feature import HashingTF,StopWordsRemover,IDF,Tokenizer

In [15]:
%%time
sc.stop()
sc = SparkContext("local[2]", "Application")
sqlContext = SQLContext(sc)

Wall time: 554 ms


In [22]:
%%time
path = "C:\\Users\\lenovo\\Desktop\\PROJECTS\\SPARK\\newsgroup_data\\mini_newsgroups\\*"
newsGroupRowData = sc.wholeTextFiles(path)
print("Number of documents read in is:", newsGroupRowData.count())

Number of documents read in is: 1854
Wall time: 18.5 s


In [23]:
newsGroupRowData.takeSample(False, 1, 10)

[('file:/C:/Users/lenovo/Desktop/PROJECTS/SPARK/newsgroup_data/mini_newsgroups/176881',
  'Xref: cantaloupe.srv.cs.cmu.edu talk.politics.misc:176881 talk.abortion:118611\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!gatech!udel!wupost!uunet!olivea!sgigate!odin!fido!zola!eno.esd.sgi.com!cj\nFrom: cj@eno.esd.sgi.com (C.J. Silverio)\nNewsgroups: talk.politics.misc,talk.abortion\nSubject: Re: Tieing Abortion to Health Reform -- Is Clinton Nuts?\nMessage-ID: <h3lnrb8@zola.esd.sgi.com>\nDate: 5 Apr 93 20:24:11 GMT\nReferences: <1pdbocINNg8g@geraldo.cc.utexas.edu> <1pdi47$3mo@agate.berkeley.edu> <sandvik-310393202727@sandvik-kent.apple.com> <1993Apr1.181711.9172@ringer.cs.utsa.edu> <1993Apr2.230831.18332@wdl.loral.com> <C4z3xw.3EF@news.cso.uiuc.edu>\nSender: news@zola.esd.sgi.com (Net News)\nReply-To: cj@sgi.com\nFollowup-To: talk.politics.misc\nOrganization: SGI Developer Docudramas\nLines: 27\n\n\nIn article <C4z3xw.3EF@news.cso.uiuc.edu>, p

In [30]:
%%time 
filepaths = newsGroupRowData.map(lambda filepath : filepath[0])
print(filepaths.takeSample(False, 5, 10))

['file:/C:/Users/lenovo/Desktop/PROJECTS/SPARK/newsgroup_data/mini_newsgroups/20896', 'file:/C:/Users/lenovo/Desktop/PROJECTS/SPARK/newsgroup_data/mini_newsgroups/54171', 'file:/C:/Users/lenovo/Desktop/PROJECTS/SPARK/newsgroup_data/mini_newsgroups/59458', 'file:/C:/Users/lenovo/Desktop/PROJECTS/SPARK/newsgroup_data/mini_newsgroups/176881', 'file:/C:/Users/lenovo/Desktop/PROJECTS/SPARK/newsgroup_data/mini_newsgroups/55060']
Wall time: 32.7 s


In [34]:
%%time 
text = newsGroupRowData.map(lambda filepath :  filepath[1])
print(text.takeSample(False, 1, 10))

['Xref: cantaloupe.srv.cs.cmu.edu talk.politics.misc:176881 talk.abortion:118611\nPath: cantaloupe.srv.cs.cmu.edu!crabapple.srv.cs.cmu.edu!fs7.ece.cmu.edu!europa.eng.gtefsd.com!gatech!udel!wupost!uunet!olivea!sgigate!odin!fido!zola!eno.esd.sgi.com!cj\nFrom: cj@eno.esd.sgi.com (C.J. Silverio)\nNewsgroups: talk.politics.misc,talk.abortion\nSubject: Re: Tieing Abortion to Health Reform -- Is Clinton Nuts?\nMessage-ID: <h3lnrb8@zola.esd.sgi.com>\nDate: 5 Apr 93 20:24:11 GMT\nReferences: <1pdbocINNg8g@geraldo.cc.utexas.edu> <1pdi47$3mo@agate.berkeley.edu> <sandvik-310393202727@sandvik-kent.apple.com> <1993Apr1.181711.9172@ringer.cs.utsa.edu> <1993Apr2.230831.18332@wdl.loral.com> <C4z3xw.3EF@news.cso.uiuc.edu>\nSender: news@zola.esd.sgi.com (Net News)\nReply-To: cj@sgi.com\nFollowup-To: talk.politics.misc\nOrganization: SGI Developer Docudramas\nLines: 27\n\n\nIn article <C4z3xw.3EF@news.cso.uiuc.edu>, parker@ehsn21.cen.uiuc.edu writes:\n| I like the way people call it "cruel and unusual pun

In [38]:
%%time
id = filepaths.map(lambda filepath: filepath.split("//")[-1])
print(id.take(5))

["Newsgroups: comp.os.ms-windows.misc\nPath: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!caen!batcomputer!munnari.oz.au!titan!pcies2.trl.OZ.AU!d.gellert\nFrom: d.gellert@trl.oz.au (Dennis Gellert)\nSubject: Re: Why is my mouse so JUMPY? (MS MOUSE)\nMessage-ID: <d.gellert.14.0@trl.oz.au>\nLines: 25\nSender: root@trl.oz.au (System PRIVILEGED Account)\nOrganization: Telecom Research\nReferences: <_c$@byu.edu> <1993Apr23.140123.5018@cti.com>\nDate: Tue, 27 Apr 1993 23:02:39 GMT\n\nIn article <1993Apr23.140123.5018@cti.com> rlister@cti.com (Russell Lister) writes:\n>From: rlister@cti.com (Russell Lister)\n>Subject: Re: Why is my mouse so JUMPY? (MS MOUSE)\n>Date: Fri, 23 Apr 1993 14:01:23 GMT\n\n>ecktons@ucs.byu.edu (Sean Eckton) writes:\n\n>>I have a Microsoft Serial Mouse and am using mouse.com 8.00 (was using 8.20 \n>>I think, but switched to 8.00 to see if it was any better).  Vertical motion \n>>is nice and smooth, b

In [41]:
%%time
topics = filepaths.map(lambda filepath: filepath.split("\\")[-2])
print(topics.take(5))
print(topics.distict().take(20))

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 14.0 failed 1 times, most recent failure: Lost task 0.0 in stage 14.0 (TID 24, localhost, executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 177, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 172, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 268, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\Users\lenovo\Anaconda3\lib\site-packages\pyspark\rdd.py", line 1339, in takeUpToNumLeft
    yield next(iterator)
  File "<timed exec>", line 1, in <lambda>
IndexError: list index out of range

	at org.apache.spark.api.python.PythonRunner$$anon$1.read(PythonRDD.scala:193)
	at org.apache.spark.api.python.PythonRunner$$ano